In [9]:
from dotenv import load_dotenv
from langchain_openai import OpenAIEmbeddings, ChatOpenAI
from langchain_chroma import Chroma
from langchain_text_splitters import RecursiveCharacterTextSplitter
from langchain_community.document_loaders import Docx2txtLoader


## 1. 문서 내용 읽기고 분할 #################################################
loader = Docx2txtLoader('law_1.docx')

text_splitter = RecursiveCharacterTextSplitter(
    chunk_size=1500,
    chunk_overlap=200,
)

document_list = loader.load_and_split(text_splitter=text_splitter)

## 2. 임베딩 -> 백터 데이터베이스에 저장 ####################################
## 2.1. 환경변수 읽어오기
load_dotenv()

## 2.2. 임베딩 모델 지정  
embedding = OpenAIEmbeddings(model="text-embedding-3-large")

## 2.3 벡터 데이터베이스에 저장
## [방법 1] in memory
# database = Chroma.from_documents(
#     documents=document_list,
#     embedding=embedding,
# )


In [ ]:

## [방법 2] 로컬에 저장
# database = Chroma.from_documents(
#     documents=document_list,
#     embedding=embedding,
#     persist_directory='./chroma',
#     collection_name='chroma-law',
# )



In [11]:
## 로컬에 저장된 임베딩 데이터 가져오기
database = Chroma(
    collection_name='chroma-law',
    persist_directory='./chroma',
    embedding_function=embedding,
)
database

In [12]:
## 3. 질문이 있으면, 벡터 데이터베이스에서 유사도 검색 ######################
# 3.1. 사용자 질문
# query = '전세사기피해자 금융지원에 대해 설명해주세요.'
query = '전세사기피해자로 인정받기 위한 조건은?'

## 3.2. 벡터 데이터베이스에서 유사도 검색
retrieved_docs = database.similarity_search(query=query, k=3)

## 4. 유사도 검색으로 가져온 문서를 LLM에 질문과 같이 전달 ###################
## 4.1. 프롬프트 작성  
prompt = '''
[identity]
- 당신은 전세사기피해 법률 전문가입니다.
-[context]를 참고하여 사용자의 질문에 답변해주세요.

[context]
{retrieved_docs}

Question : {query}
'''

## 4.2. 프롬프트 변수에 값 설정
formatted_prompt = prompt.format(retrieved_docs=retrieved_docs, query=query)

## 4.3. LLM 모델 생성(ChatOpenAI 인스턴스 생성)
llm = ChatOpenAI(model='gpt-4o')

## 4.4. LLM 모델에 질문과 검색된 문서를 보냄  
ai_message = llm.invoke(formatted_prompt)
ai_message

AIMessage(content='전세사기피해자로 인정받기 위한 조건은 다음과 같습니다:\n\n1. **주택 임대차보호 요건 충족**: 「주택임대차보호법」 제3조에 따라 주택의 인도와 주민등록을 완료하고, 임대차계약증서상의 확정일자를 갖춰야 합니다. 임차권등기를 마쳤거나 전세권이 설정된 경우도 포함됩니다.\n\n2. **임차보증금 요건**: 임차인의 임차보증금이 5억원 이하이어야 합니다. 다만, 전세사기피해지원위원회가 여건을 고려하여 보증금을 2억원 범위에서 상향 조정할 수 있습니다.\n\n3. **임대인의 경제적 상태**: 임대인의 파산 또는 회생절차 개시, 임차주택의 경매 또는 공매절차의 개시, 임차인의 집행권원 확보 등으로 인해 2인 이상의 임차인이 임차보증금반환채권의 변제를 받지 못하는 피해가 발생했거나 발생할 것이 예상되어야 합니다.\n\n4. **임대인의 의심 사유**: 임대인에 대한 수사 개시, 기망, 임차보증금을 반환할 능력이 없는 자에 대한 주택의 양도 등의 이유로 임대인이 임차보증금반환채무를 이행하지 않을 의도가 있다는 의심할 만한 상당한 이유가 있어야 합니다.\n\n또한, 임차인이 임차보증금 반환을 위한 보증 또는 보험에 가입했거나 보증금 전액이 최우선 변제가 가능한 경우 등은 제외됩니다.', additional_kwargs={'refusal': None}, response_metadata={'token_usage': {'completion_tokens': 366, 'prompt_tokens': 3132, 'total_tokens': 3498, 'completion_tokens_details': {'accepted_prediction_tokens': 0, 'audio_tokens': 0, 'reasoning_tokens': 0, 'rejected_prediction_tokens': 0}, 'prompt_tokens_details': {'audio_tokens': 0, 'cached_tokens': 3072}}, 'model_name': 'g